In [1]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
w = Window().orderBy(f.lit('A'))

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/19 23:46:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/19 23:46:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/19 23:46:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
#Leitura de CSV
df0 = spark.read.csv('work/Clientes.csv',sep=';',header='false');

In [4]:
df1 = df0.select(df0._c0.cast("integer").alias('CODIGO'),\
                        df0._c1.cast("string").alias('NOME'),\
                        df0._c2.cast("string").alias('TELEFONE'),\
                        df0._c3.cast("string").alias('EMAIL'),\
                        # df0._c4.cast("string").alias('NASCIMENTO'),\
                        # df0._c5.cast("string").alias('ANIVERSARIANTE'),\
                        # df0._c6.cast("integer").alias('IDADE'),\
                        # df0._c7.cast("string").alias('IDADE'),\
                        df0._c8.cast("string").alias('ORIGEM'),\
                        df0._c9.cast("string").alias('CADASTRO'),\
                        f.substring(df0._c9,1,2).cast("int").alias('DIA_CAD'),\
                        f.substring(df0._c9,4,2).cast("int").alias('MES_CAD'),\
                        f.substring(df0._c9,7,4).cast("int").alias('ANO_CAD'),\
                        df0._c10.cast("string").alias('UNIDADE'),\
                        f.regexp_replace(f.upper(df0._c1), " ", "").alias('NOMEUPPER'))\
                        .withColumn('DATAINGESTAO', f.lit('20220614'))
print(f'Clientes Formatados : {df0.count()}')

Clientes Formatados : 302


In [5]:
df2 = df1.dropDuplicates(['NOMEUPPER']).drop('NOMEUPPER')

In [6]:
df3 = df2.dropna(how='all', subset='CODIGO')
print(f'Clientes Higienizado : {df3.count()}')

Clientes Higienizado : 290


In [7]:
df_clientes = df3.select(f.row_number().over(w).alias('ID'),'*')

In [8]:
df_clientes.write.format("parquet").mode("overwrite").parquet('work/clientes')

22/06/19 23:47:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:36 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [9]:
df_clientes.groupBy('DIA_CAD','MES_CAD','ANO_CAD').agg(f.count('CODIGO').alias('Total')).show()

+-------+-------+-------+-----+
|DIA_CAD|MES_CAD|ANO_CAD|Total|
+-------+-------+-------+-----+
|     10|      4|   2022|    3|
|     12|      4|   2022|   10|
|     14|      4|   2022|   20|
|      9|      4|   2022|   13|
|      5|      4|   2022|   11|
|     13|      4|   2022|   20|
|     23|      4|   2022|   13|
|     15|      4|   2022|    4|
|      6|      4|   2022|    9|
|      8|      4|   2022|    8|
|     21|      4|   2022|    6|
|      2|      4|   2022|   16|
|      1|      4|   2022|   18|
|     16|      4|   2022|   16|
|      4|      4|   2022|   10|
|     19|      4|   2022|    5|
|     20|      4|   2022|    5|
|     27|      4|   2022|    7|
|     28|      4|   2022|    9|
|     17|      4|   2022|    8|
+-------+-------+-------+-----+
only showing top 20 rows



In [10]:
df = spark.read.parquet('work/clientes')

In [11]:
df_clientes.show(n=10000)

22/06/19 23:47:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/19 23:47:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+------+--------------------+---------------+--------------------+-------------------+----------+-------+-------+-------+--------------------+------------+
| ID|CODIGO|                NOME|       TELEFONE|               EMAIL|             ORIGEM|  CADASTRO|DIA_CAD|MES_CAD|ANO_CAD|             UNIDADE|DATAINGESTAO|
+---+------+--------------------+---------------+--------------------+-------------------+----------+-------+-------+-------+--------------------+------------+
|  1|  9316|  Adiana Silva-ifood|           null|                null|         Indefinido|07/04/2022|      7|      4|   2022|Inverno D´Italia ...|    20220614|
|  2|  9374|Adriele Vitoria- ...|           null|                null|         Indefinido|13/04/2022|     13|      4|   2022|Inverno D´Italia ...|    20220614|
|  3|  9291| Alanis Franco-IFOOD|           null|                null|         Indefinido|04/04/2022|      4|      4|   2022|Inverno D´Italia ...|    20220614|
|  4|  9350|Alan Marisley Fer...|       